# Backpropagation - omówienie warstw

Dla każdego elementu sieci, który będziemy używać, omówimy:
* *forward pass*,
* *backward pass*,
* powyższe w wersji batchowej (za chwilę się przekonamy, co to znaczy).

### Warstwa Dense

<img src="figures/L10/dense.png" width=400>

Zakładamy, że warstwa Dense ma rozmiary oznaczone input_size oraz output_size (w powyższym przykładzie odpowiednio 5 i 3). Wewnątrz warstwy zapamiętujemy macierz wag $W$ o rozmiarze $(\mathrm{input\_size} + 1, \mathrm{output\_size})$. Pierwszy wiersz macierzy $W$ przechowuje wagi biasu (krawędzie łączące się z węzłami oznaczanymi jedynką), dlatego dodajemy go automatycznie.

#### *Forward pass*

Na wejściu dostajemy:
* output poprzedniej warstwy $o^j$ - poziomy wektor o długości input_size,
* (w wersji batchowej) batch outputów poprzedniej warstwy $X^j$ - macierz o rozmiarach (batch_size, input_size).

Najpierw doklejamy jedynkę z lewej strony outputu poprzedniej warstwy:

$$o^j := [1,o^j]$$
$$X^j := \mathrm{hstack}(\mathrm{ones}(X^j.\mathrm{shape}[0],1), X^j)$$

zapamiętujemy zmodyfikowane wersje $o^j$ (w wersji batchowej $X^j$), będą one potrzebne do przeprowadzenia *backward pass*.

Następnie obliczamy faktyczny output:

$$o^{j+1} = o^jW$$
$$X^{j+1} = X^jW$$

#### *Backward pass*

Na wejściu dostajemy:
* $\frac{\partial L}{\partial o^{j+1}}$ - poziomy wektor pochodnych cząstkowych długości output_size,
* (w wersji batchowej) $\frac{\partial L}{\partial X^{j+1}}$ - batch pochodnych cząstkowych o rozmiarach (batch_size, output_size).

Obliczamy pochodne cząstkowe:
$$\frac{\partial L}{\partial W} = (o^j)^T\frac{\partial L}{\partial o^{j+1}}$$
$$\frac{\partial L}{\partial o^j} = \frac{\partial L}{\partial o^{j+1}}W$$

w wersji batchowej:
$$\frac{\partial L}{\partial W} = (X^j)^T\frac{\partial L}{\partial X^{j+1}}$$
$$\frac{\partial L}{\partial X^j} = \frac{\partial L}{\partial X^{j+1}}W$$

Do następnej warstwy propagujemy tylko $\frac{\partial L}{\partial o^j}[1:]$ (w wersji batchowej $\frac{\partial L}{\partial X^j}[:,1:]$) - dlaczego?


### Warstwa ReLU

<img src="figures/L10/relu.png" width=400>

Zakładamy, że warstwa ReLU ma rozmiar oznaczony layer_size == input_size == output_size (w powyższym przykładzie 4).  Warstwa aktywacji nie ma parametrów.

#### *Forward pass*

Na wejściu dostajemy:
* output poprzedniej warstwy $o^j$ - poziomy wektor o długości layer_size,
* (w wersji batchowej) batch outputów poprzedniej warstwy $X^j$ - macierz o rozmiarach (batch_size, layer_size).

Następnie obliczamy output (element-wise):

$$o^{j+1} = max(0, o^j)$$
$$X^{j+1} = max(0, X^j)$$


#### *Backward pass*

Na wejściu dostajemy:
* $\frac{\partial L}{\partial o^{j+1}}$ - poziomy wektor pochodnych cząstkowych długości layer_size,
* (w wersji batchowej) $\frac{\partial L}{\partial X^{j+1}}$ - batch pochodnych cząstkowych o rozmiarach (batch_size, layer_size).

Obliczamy pochodne cząstkowe:
$$\frac{\partial L}{\partial o^j} = \frac{\partial L}{\partial o^{j+1}} \circ \mathbb{1}_{o^j>0}$$

w wersji batchowej:
$$\frac{\partial L}{\partial X^j} = \frac{\partial L}{\partial X^{j+1}} \circ \mathbb{1}_{X^j>0}$$

gdzie:
* $\mathbb{1}_{M>0}$ to macierz, która ma jedynki tam, gdzie elementy $M$ są większe od zera, a zera w pozostałych przypadkach,
* $\circ$ to mnożenie macierzy element-wise.

### Warstwa Sigmoid

<img src="figures/L10/sigmoid.png" width=400>

Zakładamy, że warstwa Sigmoid ma rozmiar oznaczony layer_size == input_size == output_size (w powyższym przykładzie 4).  Warstwa aktywacji nie ma parametrów.

#### *Forward pass*

Na wejściu dostajemy:
* output poprzedniej warstwy $o^j$ - poziomy wektor o długości layer_size,
* (w wersji batchowej) batch outputów poprzedniej warstwy $X^j$ - macierz o rozmiarach (batch_size, layer_size).

Następnie obliczamy output (element-wise):

$$o^{j+1} = \sigma(o^j)$$
$$X^{j+1} = \sigma(X^j)$$

gdzie:

$$\sigma(x) = \frac{1}{1 + e^{-x}}$$


#### *Backward pass*

Na wejściu dostajemy:
* $\frac{\partial L}{\partial o^{j+1}}$ - poziomy wektor pochodnych cząstkowych długości layer_size,
* (w wersji batchowej) $\frac{\partial L}{\partial X^{j+1}}$ - batch pochodnych cząstkowych o rozmiarach (batch_size, layer_size).

Obliczamy pochodne cząstkowe:
$$\frac{\partial L}{\partial o^j} = \frac{\partial L}{\partial o^{j+1}} \circ \sigma(o^j) \circ (1-\sigma(o^j))$$

w wersji batchowej:
$$\frac{\partial L}{\partial X^j} = \frac{\partial L}{\partial X^{j+1}} \circ \sigma(X^j) \circ (1-\sigma(X^j))$$


### Mean Squared Error


$$L(\mathrm{y_{true}}, \mathrm{y_{pred}}) = L(t, \hat y) = \frac12(t - \hat y)^2$$
$$L(\mathbf{y_{true}}, \mathbf{y_{pred}}) = L(\mathbf{t}, \mathbf{\hat y}) = \frac{1}{\mathrm{len}(\mathbf{t})} \frac12\|t - \hat y\|_2^2$$

#### *Forward pass*

Na wejściu dostajemy:
* $\hat y$ - predykcję sieci neuronowej (wektor o długości output_size ostatniej warstwy, w naszych przykładach będzie to pojedyncza liczba),
* $t$ - prawdziwą etykietę (taki sam rozmiar, jak $\hat y$).

w wersji batchowej:
* batch predykcji sieci neuronowej $\mathbf{\hat y}$ - macierz o rozmiarach (batch_size, output_size),
* batch prawdziwych etykiet $\mathbf{t}$ - macierz o takich samych rozmiarach, jak $\mathbf{\hat y}$.

Następnie obliczamy output (koszt) zgodnie z powyższymi wzorami.

#### *Backward pass*

Jest to pierwszy *backward pass*, na wejściu nie dostajemy nic.

Obliczamy pochodne cząstkowe:
$$\frac{\partial L}{\partial \hat y} = \hat y - t$$

w wersji batchowej:
$$\frac{\partial L}{\partial \mathbf{\hat y}} = \frac{1}{\mathrm{len}(\mathbf{t})} (\mathbf{\hat y} - \mathbf{t})$$


### (mean) Cross entropy

$$ L(\mathrm{y_{true}}, \mathrm{y_{pred}}) = L(t, \hat y) = - t \log \hat y - (1 - t) \log (1 - \hat y ) $$
$$ L(\mathbf{y_{true}}, \mathbf{y_{pred}}) = L(\mathbf{t}, \mathbf{\hat y}) =\frac{1}{\mathrm{len}(\mathbf{t})} \sum_{j=1}^{\mathrm{len}(\mathbf{t})} - \mathbf{t}_j \log \mathbf{\hat y}_j - (1 - \mathbf{t}_j) \log (1 - \mathbf{\hat y}_j ) $$

#### *Forward pass*

Na wejściu dostajemy:
* $\hat y$ - predykcję sieci neuronowej (wektor o długości output_size ostatniej warstwy, w naszych przykładach będzie to pojedyncza liczba),
* $t$ - prawdziwą etykietę (taki sam rozmiar, jak $\hat y$).

w wersji batchowej:
* batch predykcji sieci neuronowej $\mathbf{\hat y}$ - macierz o rozmiarach (batch_size, output_size),
* batch prawdziwych etykiet $\mathbf{t}$ - macierz o takich samych rozmiarach, jak $\mathbf{\hat y}$.

Następnie obliczamy output (koszt) zgodnie z powyższymi wzorami.

#### *Backward pass*

Jest to pierwszy *backward pass*, na wejściu nie dostajemy nic.

Obliczamy pochodne cząstkowe:
$$\frac{\partial L}{\partial \hat y} = -\frac{t}{\hat y} + \frac{1-t}{1-\hat y} $$

w wersji batchowej (wszystkie działania wykonujemy element-wise):
$$\frac{\partial L}{\partial \mathbf{\hat y}} = \frac{1}{\mathrm{len}(\mathbf{t})}(-\frac{\mathbf{t}}{\mathbf{\hat y}} + \frac{1-\mathbf{t}}{1-\mathbf{\hat y}})$$


### Pochodne, których nie liczymy, a moglibyśmy

Nic nie stoi na przeszkodzie, żebyśmy policzyli:
* $\frac{\partial L}{\partial y_{true}}$, bezpośrednio ze wzoru na $L$, podobnie jak w sekcji "Wpływ neuronów z najwyższej warstwy",
* $\frac{\partial L}{\partial o^0} = \frac{\partial L}{\partial \mathbf{x}}$, jako ostatni krok w *backward pass*.

Pochodne te reprezentują siłę wpływu prawdziwej etykiety $y$ oraz cech $\mathbf{x}$ na wartość funkcji kosztu.

Pytanie kontrolne - dlaczego nie liczymy tych pochodnych?